In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import cv2
tf.__version__

'2.0.0'

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, 'Not enough GPU hardware devices available'
for physical_device in physical_devices:
    tf.config.experimental.set_memory_growth(physical_device, True)

In [3]:
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.losses import *
from tensorflow.keras import activations

In [4]:
from STNFunction import *
from losses import *
from Generator import *
from Discriminator import *
from utils.image_landmark import *

In [5]:
from utils.coco2017_load import get_files_and_landmarks_list

In [6]:
BATCH_SIZE = 8 
EPOCHS = 100
NUMIMAGES = 1000
BUFFER_SIZE = 20

In [7]:
lambda_cls = 0.5
lambda_flow = 0.1
lambda_mask = 0.1
lambda_landmark = 10
lambda_reco = 5

In [8]:
import pathlib
import json
import os.path
datasetPath = pathlib.Path("../Dataset/coco2017")
filenames, landmarks = get_files_and_landmarks_list(datasetPath/"annotations"/"skeleton_train2017_related.json",\
                                                datasetPath/"skeleton_train2017")

In [13]:
IMG_SIZE = (280,140,3)
def map_coco2017_skeleton(filename,landmark):
    img = tf.io.read_file(tf.convert_to_tensor(filename))
    img = tf.io.decode_jpeg(img)
    img = tf.image.convert_image_dtype(img,tf.float32)
    img = tf.image.resize_with_pad(img,IMG_SIZE[0],IMG_SIZE[1],method=tf.image.ResizeMethod.BILINEAR,antialias=False)
    # landmark = landmark_resize_with_pad(landmark,img.shape[0],img.shape[1],IMG_SIZE[0],IMG_SIZE[1])
    [newlandmark] = tf.py_function(func=landmark_resize_with_pad,
                                       inp=[landmark,img,IMG_SIZE[0],IMG_SIZE[1]],
                                       Tout=[tf.float32])
    # newlandmark = landmark
    return img,newlandmark

In [14]:
ds = tf.data.Dataset.from_tensor_slices((filenames, landmarks))
ds = ds.map(map_coco2017_skeleton) # ,num_parallel_calls=tf.data.experimental.AUTOTUNE)
Ax_ds = ds.shard(2,0).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
By_ds = ds.shard(2,1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [16]:
for (a,b) in Ax_ds:
    print(tf.reduce_mean(a))
    print(b)
    break

tf.Tensor(0.26617125, shape=(), dtype=float32)
tf.Tensor(
[[[1.7928572  0.17857143]
  [1.8785714  0.15      ]
  [1.7428571  0.12857144]
  [1.9285715  0.175     ]
  [1.6071428  0.13928571]
  [2.0285714  0.32142857]
  [1.3214285  0.27142859]
  [2.2714286  0.35714287]
  [0.98571426 0.42142856]
  [2.3214285  0.38214287]
  [0.87142855 0.6142857 ]
  [1.3642857  0.7214286 ]
  [0.94285715 0.65      ]
  [1.3142858  0.825     ]
  [0.67142856 0.78571427]
  [0.9357143  1.2714286 ]
  [0.3857143  1.1714286 ]]

 [[0.1        0.02857143]
  [0.10714286 0.025     ]
  [0.08571429 0.025     ]
  [0.12857144 0.025     ]
  [0.07857143 0.025     ]
  [0.15       0.05357143]
  [0.05714286 0.05357143]
  [0.19285715 0.09642857]
  [0.02142857 0.09285714]
  [0.14285715 0.12142857]
  [0.02857143 0.11785714]
  [0.12142857 0.13214286]
  [0.07142857 0.13214286]
  [0.12857144 0.19642857]
  [0.06428572 0.19642857]
  [0.13571429 0.24642856]
  [0.07142857 0.24642856]]

 [[0.15       0.03214286]
  [0.16428572 0.02857143]
  

In [ ]:
generator = generator_model(IMG_SIZE,BATCH_SIZE)

In [ ]:
discriminator = AttrbuteMultiscalePatchDisc()

In [ ]:
flow_loss_func = flow_loss()
lsgan_loss_func = lsgan_loss()
g_LrDecay = tf.keras.optimizers.schedules.ExponentialDecay(0.002,decay_steps=3000,
                                                          decay_rate=0.95,
                                                          staircase=True)
d_LrDecay = tf.keras.optimizers.schedules.ExponentialDecay(0.002,decay_steps=1800,
                                                          decay_rate=0.95,
                                                          staircase=True)
generator_optimizer = tf.keras.optimizers.Adam(0.002)
discriminator_optimizer = tf.keras.optimizers.Adam(d_LrDecay)

In [ ]:
def train_step(Ax,By,landmark_Ax,landmark_By,epoch):
    with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
        g_items = generator(Ax,By,epoch,True)
        # 1. TV_reg loss
        G_flow_loss = 0.0
        for flow in g_items['flows']:
            G_flow_loss += flow_loss_func.totalVariation_loss(flow)
        G_flow_loss = lambda_flow * tf.cast(G_flow_loss,'float32')
        # 2. landmark loss (By_flow)
        G_land_loss = flow_loss_func.landmark_loss(landmark_By,landmark_Ax,g_items['flows'][0])
        G_land_loss += flow_loss_func.landmark_loss(landmark_Ax,landmark_By,g_items['flows'][1])
        G_land_loss = lambda_landmark * tf.cast(G_land_loss,'float32')
        # 3. recon loss
            # 3-1. mask
        G_mask_loss = 0.0
        for mask in g_items['masks']:
            G_mask_loss += recon_loss(mask,tf.zeros_like(mask))
        G_mask_loss = lambda_mask * tf.cast(G_mask_loss,'float32')
            # 3-2. add-removal recon
        G_rcon_loss = recon_loss(By,g_items['fakeBx_to_By']) + recon_loss(Ax,g_items['fakeAy_to_Ax'])
        G_rcon_loss = lambda_reco * tf.cast(G_rcon_loss,'float32')
        #-------------------------------------------
        
        # 4. GAN loss
        D_fake_loss = 0.0
        D_real_loss = 0.0
        GAN_loss = 0.0
        G_cls_loss = 0.0
        D_cls_loss = 0.0
        # -- For Generator
        for itemname in ['fake_Ay','fakeAy_to_Ax','fakeBx_to_By','fake_Bx']:
            item = g_items[itemname]
            _, pred, attr_pred = discriminator(item)
            for pred_i in pred:
                D_fake_loss += lsgan_loss_func.loss_func(pred_i, False)
                GAN_loss += lsgan_loss_func.loss_func(pred_i, True)
            
            for attr in attr_pred:
                if itemname in 'fake_Ay' or itemname in 'By':
                    attr_label = tf.ones_like(attr)
                else:
                    attr_label = tf.zeros_like(attr)
                G_cls_loss += lambda_cls * tf.cast(cls_loss(attr,attr_label),'float32')
        D_fake_loss = 0.5 * D_fake_loss
        # -- For Real Data
        for (img,hasAttr) in [(Ax,False),(By,True)]:
            _, pred, attr_pred = discriminator(img)
            for pred_i in pred:
                D_real_loss += lsgan_loss_func.loss_func(pred_i, True)
            for attr in attr_pred:
                if hasAttr:
                    label = tf.ones_like(attr)
                else:
                    label = tf.zeros_like(attr)
                D_cls_loss += lambda_cls * tf.cast(cls_loss(attr,label),'float32')
        # -- CAST
        D_fake_loss = tf.cast(D_fake_loss,'float32')
        D_real_loss = tf.cast(D_real_loss,'float32')
        GAN_loss = tf.cast(GAN_loss,'float32')
        
        DIS_loss = D_fake_loss+D_real_loss
        flow_loss = G_flow_loss+G_land_loss
        
        G_loss = G_rcon_loss+G_cls_loss+GAN_loss+flow_loss
        D_loss = DIS_loss+D_cls_loss
    gradients_of_generator = g_tape.gradient(G_loss,generator.trainable_variables)
    
    gradients_of_discriminator = d_tape.gradient(D_loss,discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator,discriminator.trainable_variables))
    return_items = {}
    return_items["G_flow_loss"] = G_flow_loss
    return_items["G_land_loss"] = G_land_loss
    return_items["G_mask_loss"] = G_mask_loss
    return_items["G_rcon_loss"] = G_rcon_loss
    return_items["G_cls_loss"] = G_cls_loss
    return_items["GAN_loss"] = GAN_loss
    return_items["D_fake_loss"] = D_fake_loss
    return_items["D_real_loss"] = D_real_loss
    return_items["D_cls_loss"] = D_cls_loss
    return_items["G_loss"] = G_loss
    return_items["D_loss"] = D_loss
    return return_items

In [ ]:
# -- for whole train step
lossnames = ["G_flow_loss","G_land_loss","G_mask_loss","G_rcon_loss","G_cls_loss","GAN_loss",
             "D_fake_loss","D_real_loss","D_cls_loss","G_loss","D_loss"]
metrics_list = []
for itemname in lossnames:
    metrics_list.append(tf.keras.metrics.Mean(itemname, dtype=tf.float32))

for epoch in range(1,200+1):
    for (one_Ax, one_Ax_landmark), (one_By, one_By_landmark) in zip(Ax_ds, By_ds):
        # train_flownet_step(one_Ax,one_By,one_Ax_landmark,one_By_landmark,tf.cast(epoch,'float32'))
        train_items = train_step(one_Ax,one_By,one_Ax_landmark,one_By_landmark,tf.cast(epoch,'float32'))
        for (idx, itemname) in enumerate(lossnames):
            metrics_list[idx](train_items[itemname])
    print("epoch: {}".format(epoch))
    for idx,itemname in enumerate(lossnames):
        print("    {}: {:.4f}".format(itemname,metrics_list[idx].result()))
    #print("epoch: {}, G_loss: {:.4f}, D_loss: {:.4f}, flow_loss: {:.3}".format(epoch, 
    #                                                         metrics_list[9].result(),metrics_list[10].result(),
    #                                                         metrics_list[0].result()+metrics_list[1].result()))
    for metric in metrics_list:
        metric.reset_states()
    if epoch % 5 == 0:
        for (one_Ax, one_Ax_landmark), (one_By, one_By_landmark) in zip(Ax_ds, By_ds):
            g_items = generator(one_Ax,one_By,tf.cast(epoch,'float32'),False)
            save_images([one_Ax,one_By,g_items['By_warpped'].numpy(),
                         g_items['masks'][2][:,:,:,0],
                         g_items['masks'][0][:,:,:,0],
                         g_items['raw_fake_Ay'].numpy(),g_items['fake_Ay'].numpy(),g_items['residual_Ay'].numpy(),
                         g_items['fakeAy_to_Ax'].numpy(),g_items['masks'][1][:,:,:,0],
                         g_items['fake_Bx'].numpy(),g_items['fakeBx_to_By'].numpy()],
                        epoch,BATCH_SIZE,'GeoGAN1126')
            break

In [ ]:
# -- Just for flow net
lossnames = ["G_flow_loss","G_land_loss"]
metrics_list = []
for itemname in lossnames:
    metrics_list.append(tf.keras.metrics.Mean(itemname, dtype=tf.float32))
    
for epoch in range(1,400+1):
    for (one_Ax, one_Ax_landmark), (one_By, one_By_landmark) in zip(Ax_ds, By_ds):
        train_items = train_flowNet_step(one_Ax,one_By,one_Ax_landmark,one_By_landmark,tf.cast(epoch,'float32'))
        for (idx, itemname) in enumerate(lossnames):
            metrics_list[idx](train_items[itemname])
    print("epoch: {}, G_flow_loss: {}, G_land_loss: {}".format(epoch, metrics_list[0].result(),metrics_list[1].result()))
    metrics_list[0].reset_states()
    metrics_list[1].reset_states()

In [ ]:
# -- show temp image(processing)
fig = plt.figure(figsize=(75, 75))
gs = matplotlib.gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 1],
        wspace=0.0, hspace=0.0, top=0.95, bottom=0.05, left=0.1, right=0.2)
img_idx = 6
imgs = [testAx[img_idx],testBy[img_idx],g_items['fakeBx_to_By'][img_idx],g_items['masks'][1][img_idx,:,:,0]]
for idx,img in enumerate(imgs):
    ax = plt.subplot(gs[0,idx])
    ax.axis('off')
    ax.imshow(np.clip((img+1)/2,0,1))

In [ ]:
# -- show temp image(showing)
fig = plt.figure(figsize=(75, 75))
gs = matplotlib.gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 1],
        wspace=0.0, hspace=0.0, top=0.95, bottom=0.05, left=0.1, right=0.2)
img_idx = 7
imgs = [testAx[img_idx],testBy[img_idx],test_By_warpped[img_idx],test_raw_Ay[img_idx]]
for idx,img in enumerate(imgs):
    ax = plt.subplot(gs[0,idx])
    ax.axis('off')
    ax.imshow(np.clip((img+1)/2,0,1))